In [2]:
import sys
sys.path.append('../')
import importlib
import models.transunet as vit
import experiments.config as conf
import data_processing.dataset_synapse as dp 
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf 
import tensorflow_addons as tfa

In [23]:
!pwd

/home/bouzidkenza1/TransUnet/experiments


In [24]:
!cd /home/bouzidkenza1/TransUnet/data/train_npz
!ls

__init__.py  __pycache__  config.py  test-vit.ipynb  test-vm.ipynb


In [12]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
%%time
data_dir = "/home/bouzidkenza1/TransUnet/data/train_npz/"
X, y = dp.load_data(data_dir, 10)

CPU times: user 707 ms, sys: 663 ms, total: 1.37 s
Wall time: 3.58 s


In [ ]:
X.shape

In [5]:
config = conf.get_b16_none()
trans = vit.TransUnet(config)
trans.compile()

../models/utils.py:84: UserWarning: Resizing position embeddings from 24 to 32
  UserWarning,


In [ ]:
%%time
with tf.device('device:GPU:0'):
    trans.model.fit(x=X, y=y, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
1/1 [==============================] - 2s 2s/step - loss: 1.4170
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.7567
Epoch 3/3
